In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
import pandas as pd

# Загрузка датасета
data = pd.read_csv('/content/drive/MyDrive/intern_task.csv')

'''print(data.head())
print(data.info())
print(data.isnull().sum())
print(data.describe())'''



'print(data.head())\nprint(data.info())\nprint(data.isnull().sum())\nprint(data.describe())'

In [ ]:
grouped_data = data.groupby('query_id')

print("Количество уникальных query_id:", len(grouped_data))
print("Размеры каждой группы:")
print(grouped_data.size())


Количество уникальных query_id: 2000
Размеры каждой группы:
query_id
10        92
25        58
40        84
55        58
70       163
        ... 
29935     92
29950    100
29965     71
29980     70
29995     81
Length: 2000, dtype: int64


In [40]:
import pandas as pd
import matplotlib.pyplot as plt

# Загрузка данных
#data = pd.read_csv('/content/drive/MyDrive/intern_task.csv')

# Определение колонок с числовыми данными и исключение 'rank' и 'query_id'
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = numeric_cols.drop(['rank', 'query_id'])  # Удаляем 'rank' и 'query_id' из списка

# Рассчитаем IQR для каждой числовой колонки и фильтруем выбросы
for column in numeric_cols:
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Удаление выбросов
    # data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

    # Замена выбросов на медиану
    median_value = data[column].median()
    data.loc[data[column] < lower_bound, column] = median_value
    data.loc[data[column] > upper_bound, column] = median_value

In [41]:
data.head()

,rank,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
0,0,10,1.0,0.0,1.0,0.0,3.0,0.333333,0.0,0.333333,...,0.0,0.0,0.0,0.890238,8.655534,1.000000,0.017761,0.000008,1.0,0.333333
1,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.0,0.0,0.773976,23.130514,0.000000,0.027826,0.000008,44.0,14.666667
2,0,10,3.0,0.0,2.0,0.0,3.0,1.000000,0.0,0.666667,...,0.0,0.0,0.0,0.918308,13.351339,0.000000,0.014925,0.000104,22.0,7.333333
3,1,10,3.0,0.0,3.0,0.0,3.0,1.000000,0.0,1.000000,...,0.0,0.0,0.0,0.975355,18.240926,0.000000,0.053140,0.000255,8.0,2.666667
4,2,10,3.0,0.0,3.0,1.0,3.0,1.000000,0.0,1.000000,...,0.0,0.0,0.2,0.990119,31.786048,0.333333,0.046512,0.000008,24.0,8.000000


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Загрузка данных
#data = pd.read_csv('/content/drive/MyDrive/intern_task.csv')

# Определение колонок для масштабирования, исключая 'rank' и 'query_id'
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = numeric_cols.drop(['rank', 'query_id'])  # Исключаем столбцы rank и query_id

# Стандартизация
scaler_standard = StandardScaler()
data_standardized = data.copy()
data_standardized[numeric_cols] = scaler_standard.fit_transform(data[numeric_cols])

# Минимаксная нормализация
scaler_minmax = MinMaxScaler()
data_normalized = data.copy()
data_normalized[numeric_cols] = scaler_minmax.fit_transform(data[numeric_cols])

# Вывод результатов
print("Стандартизированные данные:")
print(data_standardized.head())

print("\nМинимакс нормализованные данные:")
print(data_normalized.head())


In [58]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

# Разделяем данные на тренировочные и тестовые
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Создаем DMatrix, специфичные для XGBoost структуры данных
dtrain = xgb.DMatrix(train_data.drop(['rank', 'query_id'], axis=1), label=train_data['rank'])
dtest = xgb.DMatrix(test_data.drop(['rank', 'query_id'], axis=1), label=test_data['rank'])

# Параметры модели
params = {
    'objective': 'rank:pairwise',
    'learning_rate': 0.2,
    'max_depth': 6,
    'eval_metric': 'ndcg'
}

# Обучение модели
bst = xgb.train(params, dtrain, num_boost_round=100)

In [59]:
test_data['predictions'] = bst.predict(dtest)

# Подготовка данных для расчета NDCG
grouped = test_data.groupby('query_id').apply(
    lambda x: x.sort_values('predictions', ascending=False)
).reset_index(drop=True)

# Расчет NDCG@5 для каждой группы
ndcg_scores = []
for _, group in grouped.groupby('query_id'):
    true_relevance = group['rank'].tolist()
    scores = group['predictions'].tolist()

    # Обеспечиваем, что оба списка имеют форму списка списков и проверяем длину списка
    if len(true_relevance) >= 5:
        ndcg = ndcg_score([true_relevance], [scores], k=5)
        ndcg_scores.append(ndcg)

# Средний NDCG@5 по всем группам, где возможен расчет
if ndcg_scores:
    average_ndcg = np.mean(ndcg_scores)
    print(f"Averaged NDCG@5: {average_ndcg}")
else:
    print("Недостаточно данных для расчета NDCG@5.")

Averaged NDCG@5: 0.4630334502638477
